In [36]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H3("Chandler McCarty Module Six"))),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        data=df.to_dict('records'),
        editable=True,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_rows=[],
        selected_columns=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
     html.Hr(),
     html.Div(
            id='graph-id',
            className='col s12 m6',
            style={'width': '1000px', 'height': '1000px'}
            ),
    html.Div(
            id='map-id',
            className='col s12 m6',
            style={'width': '1000px', 'height': '1000px'}
            )
])



#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

    
#FIXME Add in the code for your geolocation chart 
def update_map(viewData, selected_rows):
    
    if selected_rows is None or len(selected_rows) == 0:
        print("No row selected")                                    
        return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=15,children=[
            dl.TileLayer(id="base-layer-id")
        ])]

    dff = pd.DataFrame.from_dict(viewData)
    row = selected_rows[0]
    
    print("Data columns:", dff.columns)
    
    try:
        latitude = dff.iloc[row]['location']['latitude']
        longitude = dff.iloc[row]['location']['latitude']
        animal_name = dff.iloc[row]['animal_name']
        animal_breed = dff.iloc[row]['animal_breed']
    except Exception as e:
        print(f"Error: [e]")
    return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=15,children=[
            dl.TileLayer(id="base-layer-id")
        ])]
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=15,
               children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[latitude, longitude,
               children[
                    dl.Tooltop(animal_breed),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(animal_name)
                    ])
                ]])
            ])
        ]

app.run_server(debug=True)

Connection Successful
Dash app running on http://127.0.0.1:26994/
No row selected
No row selected
Data columns: Index(['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat',
       'location_long', 'age_upon_outcome_in_weeks'],
      dtype='object')
Error: [e]
Data columns: Index(['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat',
       'location_long', 'age_upon_outcome_in_weeks'],
      dtype='object')
Error: [e]
